#TODO:

1. Need to sort dates out so can retrieve latest market data for prediction
2. Would it help to normalize values?
3. Is trading strategy realistic?
4. Hong Kong wouldn't be available within a trading window so may need to remove (or could get current price)

# Train models

In [35]:
from datetime import datetime
import pandas as pd
from trainingData import trainingData

predictionSymbol = '^GSPC' #'^GSPC' #'^IXIC' #symbol for S&P500 (^GSPC) or NASDAQ (^IXIC) or AUS ('^AXJO')
startdate = datetime(2012,10,1)
enddate = datetime(2016,3,1)
teststartdate = datetime(2016,7,1)
delta = range(2, 5)
lags = range(2, 3)

td = trainingData()
td.load(predictionSymbol, startdate, enddate, delta, lags)

got nasdaq
got sp500
got volatility
got australia
got hkong


In [36]:
#ds = td.loadAllFromWeb(predictionSymbol, startdate, enddate)

In [37]:
#for column in ds[0].columns:
#    ds[0][column] = ds[0][column].shift(1)
#ds[0]

In [38]:
#import featureGenerator as fg
#wfeat = fg.applyRollMeanDelayedReturns(ds, delta)

In [39]:
testsize = 0.1
X_train, y_train, X_test, y_test = td.returnDataForClassification(teststartdate, testsize)

In [40]:
y_test.size

78

In [41]:
# score accuracy of each algorithm
import algorithms as alg
accuracy, rfc = alg.performRFClass(X_train, y_train, X_test, y_test)
print('RFC =', accuracy)
accuracy, knn = alg.performKNNClass(X_train, y_train, X_test, y_test)
print('KNN =', accuracy)
accuracy, svm = alg.performSVMClass(X_train, y_train, X_test, y_test)
print('SVM =', accuracy)
accuracy, ada = alg.performAdaBoostClass(X_train, y_train, X_test, y_test)
print('ADA =', accuracy)
accuracy, gtb = alg.performGTBClass(X_train, y_train, X_test, y_test)
print('GTB =', accuracy)
#accuracy = alg.performQDAClass(X_train, y_train, X_test, y_test)
#print('QDA =', accuracy)
accuracy, mlp = alg.performMLPClass(X_train, y_train, X_test, y_test, (4,5,2))
print('MLP =', accuracy)

RFC = 0.576923076923
KNN = 0.602564102564
SVM = 0.564102564103
ADA = 0.551282051282
GTB = 0.538461538462
MLP = 0.564102564103


# Evaluate for single prediction

In [8]:
# evaluate for single predictions across all algorithms
startdate = datetime(2016,10,1)
enddate = datetime(2016,10,12)
delta = range(2, 5)
lags = range(2, 3)

td = trainingData()
td.load(predictionSymbol, startdate, enddate, delta, lags)
predData = td.returnDataForPrediction()

got nasdaq
got sp500
got volatility
got australia
got hkong


In [31]:
predData = predData[:1]
predData

,OutTime2,OutRolMean2,OutTime3,OutRolMean3,OutTime4,OutRolMean4,Return_^AXJO,^AXJOTime2,^AXJORolMean2,^AXJOTime3,...,^HSITime3,^HSIRolMean3,^HSITime4,^HSIRolMean4,Return_Out2,Return_^AXJO2,Return_^IXIC2,Return_^GSPC2,Return_^VIX2,Return_^HSI2
Date,,,,,,,,,,,,,,,,,,,,,
2016-10-07,-0.002773,-0.001386,0.001511,0.000508,-0.003452,-0.000858,0.001463,-0.001386,-0.000691,0.004126,...,-0.010038,-0.003325,-0.005906,-0.00145,0.000482,0.00552,0.004983,0.004297,-0.046955,0.006902


In [30]:

#td.finalTrainingData.columns[1:].size
prediction = rfc.predict(predData)
print('rfc prediction =', prediction)
prediction = knn.predict(predData)
print('knn prediction =', prediction)
prediction = svm.predict(predData)
print('svm prediction =', prediction)
prediction = ada.predict(predData)
print('ada prediction =', prediction)
prediction = gtb.predict(predData)
print('gtb prediction =', prediction)
prediction = mlp.predict(predData)
print('mlp prediction =', prediction)

rfc prediction = [1]
knn prediction = [0]
svm prediction = [1]
ada prediction = [1]
gtb prediction = [1]
mlp prediction = [1]


# Back test

In [16]:
from datetime import datetime
import pandas as pd
from trainingData import trainingData

predictionSymbol = '^GSPC' #'^GSPC' #'^IXIC' #symbol for S&P500 (^GSPC) or NASDAQ (^IXIC) or AUS ('^AXJO')

startdate = datetime(2016,10,1)
enddate = datetime(2016,10,14)
delta = range(2, 5)
lags = range(2, 3)

td = trainingData()
td.load(predictionSymbol, startdate, enddate, delta, lags)
predData = td.returnDataForBacktesting()

got nasdaq
got sp500
got volatility
got australia
got hkong


In [25]:
from datetime import datetime
import pandas as pd
from trainingData import trainingData

predictionSymbol = '^GSPC' #'^GSPC' #'^IXIC' #symbol for S&P500 (^GSPC) or NASDAQ (^IXIC) or AUS ('^AXJO')

startdate = datetime(2016,3,2)
enddate = datetime(2016,10,14)
delta = range(2, 5)
lags = range(2, 3)

td = trainingData()
td.load(predictionSymbol, startdate, enddate, delta, lags)
predData = td.returnDataForBacktesting()

# symbol of the stock required for future plotting
symbol = 'S&P-500'

# name of the file of the output of prediction (S&P 500 in this case)
name = 'sp500-2016.csv'

# calls the best model previously saved in pickle file and runs it on the test set retutning an array of 0,1 (Down, Up) according to predicted returns
#prediction = pystocks.getPredictionFromBestModel(9, 9, 'sp500', cut, start_test, path_datasets, 'sp500_57.pickle')[0]
prediction = rfc.predict(predData)
prediction2 = mlp.predict(predData)

# dataframe of S&P 500 historical prices (saved locally from Yahho Finance)
bars = pd.read_csv(name, index_col=0, parse_dates=True)    

# subset of the data corresponding to test set
bars = pd.merge(bars, predData, how='inner', on=None, left_index=True, right_index=True).ix[:,0:6]

# initialize empty dataframe indexed as the bars. There's going to be perfect match between dates in bars and signals 
signals = pd.DataFrame(index=bars.index)

# initialize signals.signal column to zero
signals['signal'] = 0.0

# copying into signals.signal column results of prediction
signals['signal'] = prediction

signals2 = signals.copy()
signals2['signal'] = prediction2

# replace the zeros with -1 (new encoding for Down day)
signals.signal[signals.signal == 0] = -1
signals2.signal[signals2.signal == 0] = -1

signalsall = pd.merge(signals, signals2, how='inner', on=None, left_index=True, right_index=True)
signalsall = signalsall[signalsall['signal_x'] == signalsall['signal_y']].ix[:,0:1]
signalsall.columns = ['signal']

# compute the difference between consecutive entries in signals.signal. As
# signals.signal was an array of 1 and -1 return signals.positions will 
# be an array of 0s and 2s.
signals['positions'] = signals['signal'].diff()     
signalsall['positions'] = signalsall['signal'].diff()

#signals = signals.head(len(signalsall))

got nasdaq
got sp500
got volatility
got australia
got hkong


In [26]:
from portfolio import MarketIntradayPortfolio

# calling portfolio evaluation on signals (predicted returns) and bars 
# (actual returns)
portfolio = MarketIntradayPortfolio(symbol, bars, signals)

# backtesting the portfolio and generating returns on top of that 
returns = portfolio.backtest_portfolio()
returns.tail(1)

,price_diff,profit,total,returns
Date,,,,
2016-10-11,-24.620118,-12310.059,138799.8655,-0.081464


# Plot portfolio performance

In [27]:
import matplotlib.pyplot as plt

f, ax = plt.subplots(2, sharex=True)
f.patch.set_facecolor('white')
ylabel = symbol + ' Close Price in $'
bars['Close_Out'].plot(ax=ax[0], color='r', lw=3.)    
ax[0].xaxis.grid(True)
ax[0].yaxis.grid(True)
ax[0].set_ylabel(ylabel, fontsize=18)
ax[0].set_xlabel('', fontsize=18)
ax[0].legend(('Close Price S&P-500',), loc='upper left', prop={"size":18})
ax[0].set_title('S&P 500 Close Price VS Portofolio Performance (1 January 2016 - 12 October 2016)', fontsize=20, fontweight="bold")

returns['total'].plot(ax=ax[1], color='b', lw=3.)  
ax[1].xaxis.grid(True)
ax[1].yaxis.grid(True)
ax[1].set_ylabel('Portfolio value in $', fontsize=18)
ax[1].set_xlabel('Date', fontsize=18)
ax[1].legend(('Portofolio Performance. Capital Invested: 100k $. Shares Traded per day: 500+500',), loc='upper left', prop={"size":18})            
plt.tick_params(axis='both', which='major', labelsize=14)
loc = ax[1].xaxis.get_major_locator()
#loc.maxticks[DAILY] = 24

figManager = plt.get_current_fig_manager()
figManager.window.showMaximized()

plt.show()